In [1]:
%%latex 
from IPython.display import Latex

<IPython.core.display.Latex object>

# Gaussian Discriminant Analysis (GDA)
GDA is part of the algorithms called **Generative learning algorthm**. We call the so because of the fact that they try to model $p(x/y)$ and $p(y)$.
After modeling $p(y)$ (called the **class priors**) and $p(x|y)$, the  algorithm can they use Bayes rule to derive the posterior distribution on $y$ given:
$$p(y|x) = \frac{p(x|y) p(y)}{p(x)}$$
Here in case of binary classification, the denominator is given by $p(x) = p(x|y=1)p(y=1) + p(x|y=0)p(y=0)$. Actually, if we were calculating $p(y|x)$ in order to make a prediction, then we don't actually need to calculate the denominator, since $$\begin{aligned}
\underset{y}{\arg \max } p(y \mid x) &=\arg \max _{y} \frac{p(x \mid y) p(y)}{p(x)} \\
&=\arg \max _{y} p(x \mid y) p(y)
\end{aligned}$$
The particularity of the GDA model is the fact that we will assume that $p(x|y)$ is distributed according to a multivariate normal distribution.

When we have a classification problem in which the input features $x$ are continuous-valued random variables, we can then use the Gaussian Discriminant Analysis (GDA) model, which models p(x|y) using a multivariate normal distribution. The model is:
$$
\begin{aligned}
y & \sim \operatorname{Bernoulli}(\phi) \\
x \mid y=0 & \sim \mathcal{N}\left(\mu_{0}, \Sigma\right) \\
x \mid y=1 & \sim \mathcal{N}\left(\mu_{1}, \Sigma\right)
\end{aligned}
$$
Writing out the distributions, this is:
$$
\begin{aligned}
p(y) &=\phi^{y}(1-\phi)^{1-y} \\
p(x \mid y=0) &=\frac{1}{(2 \pi)^{n / 2}|\Sigma|^{1 / 2}} \exp \left(-\frac{1}{2}\left(x-\mu_{0}\right)^{T} \Sigma^{-1}\left(x-\mu_{0}\right)\right) \\
p(x \mid y=1) &=\frac{1}{(2 \pi)^{n / 2}|\Sigma|^{1 / 2}} \exp \left(-\frac{1}{2}\left(x-\mu_{1}\right)^{T} \Sigma^{-1}\left(x-\mu_{1}\right)\right)
\end{aligned}
$$
Here, the parameters of our model are $\phi, \Sigma, \mu_{0}$ and $\mu_{1}$. (Note that while there're two different mean vectors $\mu_{0}$ and $\mu_{1},$ this model is usually applied using only one covariance matrix $\Sigma$. ) The log-likelihood of the data is given by
$$
\begin{aligned}
\ell\left(\phi, \mu_{0}, \mu_{1}, \Sigma\right) &=\log \prod_{i=1}^{m} p\left(x^{(i)}, y^{(i)} ; \phi, \mu_{0}, \mu_{1}, \Sigma\right) \\
&=\log \prod_{i=1}^{m} p\left(x^{(i)} \mid y^{(i)} ; \mu_{0}, \mu_{1}, \Sigma\right) p\left(y^{(i)} ; \phi\right)
\end{aligned}
$$
By maximizing the likelihood $l$ with respect to the parameters, we find the maximum likelihood from estimate of the parameters to be: 
\begin{aligned}
\phi &=\frac{1}{m} \sum_{i=1}^{m} 1\left\{y^{(i)}=1\right\} \\
\mu_{0} &=\frac{\sum_{i=1}^{m} 1\left\{y^{(i)}=0\right\} x^{(i)}}{\sum_{i=1}^{m} 1\left\{y^{(i)}=0\right\}} \\
\mu_{1} &=\frac{\sum_{i=1}^{m} 1\left\{y^{(i)}=1\right\} x^{(i)}}{\sum_{i=1}^{m} 1\left\{y^{(i)}=1\right\}} \\
\Sigma &=\frac{1}{m} \sum_{i=1}^{m}\left(x^{(i)}-\mu_{y^{(i)}}\right)\left(x^{(i)}-\mu_{y^{(i)}}\right)^{T}
\end{aligned}

## Implementation of the GDA from scratch
In this part we are going the to implementation step by step the estimation of the parameters and use them them to compute the maximum likelihood.
We will finish it by combine all the process into an Object that we can call.

In [21]:
import numpy as np
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

In [30]:
type(y)

numpy.ndarray

### Compute the prior probability

In [31]:
def prior_probability(target):
    theta = np.array([(target==c).mean() for c in np.unique(target)])
    return theta

prior_probability(y)

array([0.33333333, 0.33333333, 0.33333333])

### Compute the estimation of the expectation

In [34]:
def estimated_means(X,target):
    mean = []
    for c in np.unique(target):
        dc = X[target==c]
        mc = dc.mean(axis = 0)
        mean.append(mc)
    mu = np.array(mean)
    return mu

estimated_means(X,y)

array([[5.006, 3.428, 1.462, 0.246],
       [5.936, 2.77 , 4.26 , 1.326],
       [6.588, 2.974, 5.552, 2.026]])

### Compute the covariance matrice of the data

In [37]:
def cov_matrix(X,target):
    Data = X.copy()
    m = estimated_means(X,target)
    for v in np.unique(target):
        Data[target==v] = Data[target==v] - m[v]
    cov = np.dot(Data.T,Data)/len(y)
    inv_cov = np.linalg.pinv(cov)
    cov = cov
    return cov

cov_matrix(X,y)

array([[0.259708  , 0.09086667, 0.164164  , 0.03763333],
       [0.09086667, 0.11308   , 0.05413867, 0.032056  ],
       [0.164164  , 0.05413867, 0.181484  , 0.041812  ],
       [0.03763333, 0.032056  , 0.041812  , 0.041044  ]])

In [23]:

class GDAnalysis(object):
    
    def __init__(self,theta = None, mu = None, cov = None,prediction = None):
        self.theat = theta
        self.mu = mu
        self.cov = cov
        self.prediction = prediction
        
    def prior_probability(self,target):
        self.theta = np.array([(target==c).mean() for c in np.unique(target)])
        return self.theta

    def estimated_means(self,X,target):
        mean = []
        for c in np.unique(target):
            dc = X[target==c]
            mc = dc.mean(axis = 0)
            mean.append(mc)
        self.mu = np.array(mean)
        return self.mu

    def cov_matrix(self,X,target):
        Data = X.copy()
        m = self.estimated_means(X,target)
        for v in np.unique(target):
            Data[target==v] = Data[target==v] - m[v]
        cov = np.dot(Data.T,Data)/len(target)
        self.inv_cov = np.linalg.pinv(cov)
        self.cov = cov
        return cov
    
    
    def gauss_density(self,x):
        """pdf of the multivariate normal distribution."""
        x_m = x - self.mu
        return np.exp(-1 * np.sum(x_m.dot(self.inv_cov) * x_m, axis=1)) * 0.5 * self.theta
    
    def fit(self,X,target):
        self.prior_probability(target) 
        self.estimated_means(X,target)
        self.cov_matrix(X,target)
        return self.theta , self.mu , self.cov
   
    def predict(self,X):
        self.prediction = np.apply_along_axis(self.gauss_density, 1, X)
        return np.argmax(self.prediction, axis=1)
    
    def accuracy(self, y_test,y_pred):
        return (y_test==y_pred).mean()

In [24]:
GDA = GDAnalysis()
GDA.prior_probability(y)

array([0.33333333, 0.33333333, 0.33333333])

In [25]:
GDA.estimated_means(X,y)

array([[5.006, 3.428, 1.462, 0.246],
       [5.936, 2.77 , 4.26 , 1.326],
       [6.588, 2.974, 5.552, 2.026]])

In [26]:
GDA.cov_matrix(X,y)

array([[0.259708  , 0.09086667, 0.164164  , 0.03763333],
       [0.09086667, 0.11308   , 0.05413867, 0.032056  ],
       [0.164164  , 0.05413867, 0.181484  , 0.041812  ],
       [0.03763333, 0.032056  , 0.041812  , 0.041044  ]])

In [27]:
GDA.inv_cov

array([[11.06481504, -5.48691938, -9.17543051,  3.4871337 ],
       [-5.48691938, 14.52436643,  2.7242858 , -9.08804849],
       [-9.17543051,  2.7242858 , 15.09102422, -9.08813896],
       [ 3.4871337 , -9.08804849, -9.08813896, 37.52283607]])

In [28]:
pre = GDA.predict(X)

In [29]:
GDA.accuracy(y, pre)

0.98